In [1]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"lightningv08","key":"5de93316c255f2bfcb82d90101ed1d07"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [2]:
!kaggle competitions download -c tutors-lessons-prices-prediction
!unzip tutors-lessons-prices-prediction.zip

tutors-lessons-prices-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  tutors-lessons-prices-prediction.zip
replace sample_submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: н
error:  invalid response [н]
replace sample_submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: н
error:  invalid response [н]
replace sample_submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: н
error:  invalid response [н]
replace sample_submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submit.csv       
replace test.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.xlsx               
replace train.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.xlsx              


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score

In [4]:
df = pd.read_excel("train.xlsx")
df_test = pd.read_excel("test.xlsx")
df.drop(["Unnamed: 0", "ФИО"], axis=1, inplace=True)
df_test.drop(["Unnamed: 0", "ФИО"], axis=1, inplace=True)
df2 = pd.concat([df, df_test], axis=0)
df2

,предмет,tutor_head_tags,tutor_rating,tutor_reviews,description,experience_desc,Education_1,Desc_Education_1,Education_2,Desc_Education_2,...,status,experience,Desc_Education_5,Desc_Education_6,Education_5,Education_6,Ученая степень 2,Ученое звание 2,categories,mean_price
0,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",5.0,2,Репетитор не предоставил о себе дополн...,NaN,Томский государственный университет им. В.В. ...,"Год окончания: 1990, Факультет: механико-мате...",NaN,NaN,...,Университетский преподаватель,31 год,NaN,NaN,NaN,NaN,NaN,NaN,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",16.666667
1,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",4.0,7,Репетитор не предоставил о себе дополн...,NaN,Вологодский государственный педагогический ун...,"Год окончания: 2017, Факультет: факультет при...",Череповецкий государственный университет,"Год окончания: 2023, Факультет: институт инфо...",...,Школьный преподаватель,8 лет,NaN,NaN,NaN,NaN,NaN,NaN,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",11.666667
2,informatika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,"Выезд за город: Балашиха-1, Балашиха-2...",Московская Школа (учитель ...,Московский городской педагогический университет,"Год окончания: 2008, Факультет: Математически...",Московский городской педагогический университ...,"Год окончания: 2014, Факультет: Безопасность ...",...,Школьный преподаватель,11 лет,NaN,NaN,NaN,NaN,NaN,NaN,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",23.333333
3,matematika,"['Школьная программа', 'Математический анализ'...",4.0,19,"Здравствуйте! Меня зовут Елизавета, о...",NaN,Санкт-Петербургский государственный университет,"Год окончания: 2021, Факультет: физический фа...",NaN,NaN,...,Частный преподаватель,5 лет,NaN,NaN,NaN,NaN,NaN,NaN,"['Школьники с 1 по 4 класс', 'Школьники с 5 по...",0.116667
4,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,"Целеустремленный, ориентирован на резу...",преподавателем ...,Приазовский государственный технический униве...,"Год окончания: 2005, Факультет: Энергетически...",NaN,NaN,...,Частный преподаватель,19 лет,NaN,NaN,NaN,NaN,NaN,NaN,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",16.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,matematika,"['Школьная программа', 'Подготовка к ОГЭ']",0.0,0,Репетитор по предметам начальной школы...,NaN,Алтайская государственная академия образовани...,"Год окончания: 2017, Факультет: психолого-пед...",Алтайская государственная педагогическая акад...,"Год окончания: 2019, Факультет: институт псих...",...,Частный преподаватель,9 лет,NaN,NaN,NaN,NaN,NaN,NaN,"['Дошкольники', 'Школьники с 1 по 4 класс', 'Ш...",NaN
1512,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,"Здравствуй, дорогой друг! Я частный ре...",Учитель математики в школе...,СГУ им. Чернышевского,"Год окончания: 2009, Факультет: Механико-мате...",NaN,NaN,...,Школьный преподаватель,14 лет,NaN,NaN,NaN,NaN,NaN,NaN,"['Школьники с 5 по 9 класс', 'Школьники с 5 по...",NaN
1513,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Легко нахожу общий язык с учениками. У...,Репетитор не предоставил информаци...,СГУ,"Год окончания: 2010, Факультет: Математически...",NaN,NaN,...,Частный преподаватель,13 лет,NaN,NaN,NaN,NaN,NaN,NaN,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",NaN
1514,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",4.0,32,Репетитор не предоставил о себе дополн...,NaN,Саратовский государственный университет имени...,"Год окончания: 2007, Факультет: математически...",NaN,NaN,...,Школьный преподаватель,12 лет,NaN,NaN,NaN,NaN,NaN,NaN,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   предмет           4548 non-null   object 
 1   tutor_head_tags   4548 non-null   object 
 2   tutor_rating      3959 non-null   float64
 3   tutor_reviews     4548 non-null   int64  
 4   description       4547 non-null   object 
 5   experience_desc   2171 non-null   object 
 6   Education_1       4548 non-null   object 
 7   Desc_Education_1  4472 non-null   object 
 8   Education_2       781 non-null    object 
 9   Desc_Education_2  777 non-null    object 
 10  Education_3       106 non-null    object 
 11  Desc_Education_3  105 non-null    object 
 12  Education_4       16 non-null     object 
 13  Desc_Education_4  16 non-null     object 
 14  Ученая степень 1  76 non-null     object 
 15  Ученое звание 1   35 non-null     object 
 16  status            4535 non-null   object 


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1516 entries, 0 to 1515
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   предмет           1516 non-null   object 
 1   tutor_head_tags   1516 non-null   object 
 2   tutor_rating      1516 non-null   int64  
 3   tutor_reviews     1516 non-null   int64  
 4   description       1515 non-null   object 
 5   experience_desc   757 non-null    object 
 6   Education_1       1516 non-null   object 
 7   Desc_Education_1  1498 non-null   object 
 8   Education_2       286 non-null    object 
 9   Desc_Education_2  285 non-null    object 
 10  Education_3       34 non-null     object 
 11  Desc_Education_3  34 non-null     object 
 12  Education_4       3 non-null      object 
 13  Desc_Education_4  3 non-null      object 
 14  Ученая степень 1  20 non-null     object 
 15  Ученое звание 1   7 non-null      object 
 16  status            1513 non-null   object 


In [7]:
df.describe()

,tutor_rating,tutor_reviews,mean_price
count,3959.000000,4548.000000,4548.000000
mean,1.895933,2.989006,16.197949
std,2.280103,8.516939,10.485108
min,0.000000,0.000000,0.041667
25%,0.000000,0.000000,10.000000
50%,0.000000,0.000000,15.000000
75%,4.000000,2.000000,20.888889
max,5.000000,152.000000,100.000000


In [8]:
df['предмет'].value_counts()

,count
предмет,
matematika,3685
informatika,863


In [9]:
df['status'].value_counts()

,count
status,
Частный преподаватель,2345
Студент,1288
Школьный преподаватель,558
Университетский преподаватель,129
"Частный преподаватель, Студент",92
"Университетский преподаватель, Школьный преподаватель",47
Аспирант или ординатор очной формы обучения,45
"Аспирант или ординатор очной формы обучения, Частный преподаватель",15
"Университетский преподаватель, Аспирант или ординатор очной формы обучения",13


In [10]:
df2['tutor_head_tags'] = df2['tutor_head_tags'].str.replace('[', '').str.replace(']', '').str.replace("'", "")
df2['tutor_head_tags'].value_counts()

,count
tutor_head_tags,
"Школьная программа, Подготовка к ЕГЭ, Подготовка к ОГЭ",2236
"Школьная программа, Подготовка к ОГЭ",1094
Школьная программа,647
"Школьная программа, Подготовка к ЕГЭ, Подготовка к ОГЭ, Подготовка к олимпиадам",439
"Подготовка к ЕГЭ, Подготовка к ОГЭ",192
...,...
"Школьная программа, Подготовка к ОГЭ, Линейная алгебра, ТФКП, Дифференциальные уравнения, Теория вероятности и мат. статистика",1
"Школьная программа, Математический анализ, Подготовка к ЕГЭ, Аналитическая геометрия, Подготовка к ОГЭ, Подготовка к олимпиадам, Дифференциальные уравнения, Теория вероятности и мат. статистика, Математическая логика, Дискретная математика",1
Дискретная математика,1


In [11]:
df2['experience'] = df2['experience'].str.replace("лет", "").str.replace("года", "").str.replace("год", "").str.replace(" ", "").fillna(0).astype(int)

In [12]:
df2.fillna({"tutor_rating" : 0, "Desc_Education_1" : ""}, inplace=True)

In [13]:
df2['categories'] = df2['categories'].str.replace('[', '').str.replace(']', '').str.replace("'", "")
df2['categories'].value_counts()

,count
categories,
"Школьники с 5 по 9 класс, Школьники с 10 по 11 класс, студенты колледжей и техникумов, абитуриенты",1680
Школьники с 5 по 9 класс,806
"Школьники с 1 по 4 класс, Школьники с 5 по 9 класс",475
"Школьники с 1 по 4 класс, Школьники с 5 по 9 класс, Школьники с 10 по 11 класс, студенты колледжей и техникумов, абитуриенты",418
"Школьники с 5 по 9 класс, Школьники с 10 по 11 класс, студенты колледжей и техникумов, абитуриенты, Студенты университетов, Взрослые кроме абитуриентов и студентов",399
...,...
"Школьники с 5 по 9 класс, Школьники с 5 по 9 класс, Школьники с 5 по 9 класс, Школьники с 10 по 11 класс, студенты колледжей и техникумов, абитуриенты, Школьники с 10 по 11 класс, студенты колледжей и техникумов, абитуриенты, Школьники с 10 по 11 класс, студенты колледжей и техникумов, абитуриенты, Взрослые кроме абитуриентов и студентов",1
"Школьники с 5 по 9 класс, Школьники с 5 по 9 класс, Школьники с 5 по 9 класс в паре",1
"Школьники с 1 по 4 класс, Школьники с 5 по 9 класс, Школьники с 5 по 9 класс в паре, Школьники с 10 по 11 класс, студенты колледжей и техникумов, абитуриенты, Школьники с 10 по 11 класс, студенты колледжей и техникумов, абитуриенты, Школьники с 10 по 11 класс, студенты колледжей и техникумов, абитуриенты в паре, Школьники с 10 по 11 класс, студенты колледжей и техникумов, абитуриенты в паре",1


In [14]:
df2['Education_1'].value_counts()

,count
Education_1,
Санкт-Петербургский государственный университет,224
Санкт-Петербургский государственный политехнический университет,124
Российский государственный педагогический университет имени А. И. Герцена,104
Репетитор не предоставил информацию об образовании,93
МГТУ им. Н.Э. Баумана,86
...,...
СГУ им. Чернышевского,1
СФ МГПУ,1
ЕГПУ,1


In [15]:
df2['educations_number'] = 4 - (df2['Education_1'].isna().astype(int) + df2['Education_2'].isna().astype(int) + df2['Education_3'].isna().astype(int) + df2['Education_4'].isna().astype(int))

In [16]:
df2['has_a_degree'] = (1 - df2['Ученая степень 1'].isna()).astype(bool)

In [17]:
import re
for index, row in df2.iterrows():
    if not row['Desc_Education_1']:
        continue
    df2.loc[df2.index == index, 'Education_1_year'] = re.search(r'\b\d+\b', row['Desc_Education_1'])[0]
df2.fillna({'Education_1_year' : 2030}, inplace=True)
df2['Education_1_year'] = df2['Education_1_year'].astype(int)
df2['Education_1_year'] = 2025 - df2['Education_1_year']

In [18]:
for col in df2:
    if ("education" in col.lower() and not "educations" in col.lower()) or "степень" in col or "звание" in col or "desc" in col.lower():
        df2.drop(col, axis=1, inplace=True)

In [19]:
df2 = pd.concat([df2, df2['предмет'].str.get_dummies(","), df2['status'].str.get_dummies(","), df2['tutor_head_tags'].str.get_dummies(", "), df2['categories'].str.get_dummies(", ")], axis=1)
df2.drop(['предмет', 'status', 'tutor_head_tags', 'categories'], axis=1, inplace=True)

In [20]:
df2

,tutor_rating,tutor_reviews,experience,mean_price,educations_number,has_a_degree,informatika,matematika,Аспирант или ординатор очной формы обучения,Студент,...,Школьники с 1 по 4 класс в паре,Школьники с 1 по 4 класс группа,Школьники с 10 по 11 класс,Школьники с 5 по 9 класс,Школьники с 5 по 9 класс в паре,Школьники с 5 по 9 класс группа,абитуриенты,абитуриенты в паре,абитуриенты группа,студенты колледжей и техникумов
0,5.0,2,31,16.666667,1,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1
1,4.0,7,8,11.666667,2,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1
2,0.0,0,11,23.333333,3,False,1,0,0,0,...,0,0,1,1,0,0,1,0,0,1
3,4.0,19,5,0.116667,1,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1
4,0.0,0,19,16.666667,1,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,0.0,0,9,NaN,2,False,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1512,0.0,0,14,NaN,1,False,0,1,0,0,...,0,0,1,1,1,0,1,1,0,1
1513,0.0,0,13,NaN,1,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1
1514,4.0,32,12,NaN,1,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1


In [21]:
df = df2.iloc[0 : 4547]
df_test = df2.iloc[4548:]

In [22]:
df

,tutor_rating,tutor_reviews,experience,mean_price,educations_number,has_a_degree,informatika,matematika,Аспирант или ординатор очной формы обучения,Студент,...,Школьники с 1 по 4 класс в паре,Школьники с 1 по 4 класс группа,Школьники с 10 по 11 класс,Школьники с 5 по 9 класс,Школьники с 5 по 9 класс в паре,Школьники с 5 по 9 класс группа,абитуриенты,абитуриенты в паре,абитуриенты группа,студенты колледжей и техникумов
0,5.0,2,31,16.666667,1,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1
1,4.0,7,8,11.666667,2,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1
2,0.0,0,11,23.333333,3,False,1,0,0,0,...,0,0,1,1,0,0,1,0,0,1
3,4.0,19,5,0.116667,1,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1
4,0.0,0,19,16.666667,1,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4542,0.0,0,6,11.111111,1,False,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
4543,4.0,11,5,16.666667,1,False,0,1,0,0,...,0,0,1,1,0,0,1,0,0,1
4544,0.0,0,30,33.333333,1,False,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4545,0.0,0,6,23.333333,1,False,1,0,0,1,...,0,0,1,1,0,0,1,0,0,1


In [23]:
x = df.drop('mean_price', axis=1)
y = df['mean_price']
X_test, X_train, y_test, y_train = train_test_split(x, y, test_size=0.3)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('R2 score:', r2_score(y_test, y_pred))

Mean Absolute Error: 7.214121633705782
Mean Squared Error: 101.21662502085744
R2 score: 0.09311289072581319


In [34]:
from sklearn.preprocessing import PolynomialFeatures
x = df.drop('mean_price', axis=1)
y = df['mean_price']
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(x)
X_test, X_train, y_test, y_train = train_test_split(x, y, test_size=0.3)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('R2 score:', r2_score(y_test, y_pred))

Mean Absolute Error: 7.09506922951151
Mean Squared Error: 97.67600241057684
R2 score: 0.13056091250331958


In [44]:
import xgboost as xgb
x = df.drop('mean_price', axis=1)
y = df['mean_price']
X_test, X_train, y_test, y_train = train_test_split(x, y, test_size=0.3)
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('R2 score:', r2_score(y_test, y_pred))

Mean Absolute Error: 7.164283946457395
Mean Squared Error: 101.59527778742446
R2 score: 0.05651309392806547


In [46]:
from sklearn.ensemble import RandomForestRegressor
x = df.drop('mean_price', axis=1)
y = df['mean_price']
X_test, X_train, y_test, y_train = train_test_split(x, y, test_size=0.3)
model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('R2 score:', r2_score(y_test, y_pred))

Mean Absolute Error: 6.8566183258276965
Mean Squared Error: 95.0018399749111
R2 score: 0.11766384492318271


In [47]:
ans = pd.DataFrame()
ans['index'] = df_test.index
ans['mean_price'] = model.predict(df_test.drop('mean_price', axis=1))
print(ans.head(10))
ans.to_csv('ans.csv', sep=',', index=False, encoding='utf-8')

   index  mean_price
0      0   22.466000
1      1    8.937611
2      2   20.423741
3      3   16.994278
4      4    2.744380
5      5   14.247625
6      6   17.183222
7      7   17.748384
8      8   19.074722
9      9   17.459074
